In [18]:
%load_ext autoreload
%autoreload 2

from wordle.wordlenp import Wordle
from environment.environment import Environment, StateYesNo, ActionVocabulary
from dqn.agent import Agent
from dqn.train import train
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# init environment
rewards = {'B':0, 'Y':1, 'G':1, 'win':10, 'lose':-10, 'step':-2}
env = Environment(
    rewards=rewards,
    wordle=Wordle(),
    state_instance=StateYesNo())

# init action
agent = Agent(
    state_size=env.state.size,
    action_size=len(env.wordle.vocabulary),
    action_constructor=ActionVocabulary)

In [20]:
train(env=env, agent=agent)


Episode  100	Average Score: -16.29	Success Rate: 0.0%	Duration: 0.9

Episode  200	Average Score: -17.13	Success Rate: 0.0%	Duration: 2.0

Episode  300	Average Score: -16.68	Success Rate: 0.0%	Duration: 46.6

Episode  400	Average Score: -15.67	Success Rate: 0.0%	Duration: 92.7


KeyboardInterrupt: 